In [1]:
from cst2coords import cst2coords
from foil_mesher import meshify
from utils import FOAM, Param
from pathlib import Path
import shutil
import time
import numpy as np

/home/gin/Desktop/ParallelComp/Code/foil_mesher.py:498: SyntaxWarning: invalid escape sequence '\ '
  "/*--------------------------------*- C++ -*----------------------------------*\ \n"
/home/gin/Desktop/ParallelComp/Code/foil_mesher.py:508: SyntaxWarning: invalid escape sequence '\*'
  "\*---------------------------------------------------------------------------*/ \n"


In [2]:
shutil.rmtree("output", ignore_errors=True)
run = "1"  # time.time()
param = Param(
    f"{run}",
    Path(f"output/{run}"),
    Path("basic_template"),
    True,
    Path("output/results/csv/results.csv"),
    fluid_velocity=np.array([99.6194698092, 8.7155742748, 0]),
)

param.run_path.mkdir(parents=True, exist_ok=True)
shutil.copytree(src=param.template_path, dst=param.run_path, dirs_exist_ok=True)
param.csv_path.parent.mkdir(exist_ok=True, parents=True)

In [3]:
# wu wl of the foil
wu = [0.65802251, 0.92974621, 1.11012945]
wl = [-0.69080258, 0.22105255, -0.50046596]
coords = cst2coords(wu, wl, 0, 50)
x, y = [], []
for i in range(len(coords)):
    x.append(coords[i][0])
    y.append(coords[i][1])
# coords = np.column_stack((x[:-1], y[::-1][:-1]))
coords = np.column_stack((x[:-1], y[:-1]))
open("test.txt", "w").write("\n".join([" ".join(map(str, x)) for x in coords]))

top_section = coords[1:][0 : 25 - 1]
bot_section = coords[1:][25:][::-1]
top_bottom_difference = top_section[:, 1] - bot_section[:, 1]

if (top_bottom_difference < 0).any():
    raise Exception("Top and bottom sections are intersecting")
# open("test.txt", "w").write("\n".join([" ".join(map(str, x)) for x in coords]))

In [4]:
FOAM.set_fluid_velocities(param.run_path, param.fluid_velocity)
meshify(Coords=coords, run_path=param.run_path)

In [5]:
block_mesh_result = FOAM.run_blockmesh(run_path=param.run_path)
check_mesh_result = FOAM.run_checkmesh(run_path=param.run_path)

In [6]:
if not (block_mesh_result and check_mesh_result):
    raise Exception("Meshing failed: " + param.run_name)

In [7]:
with open(
    param.run_path / (param.run_name + ".foam"),
    "w",
) as _:
    pass
simple_result = FOAM.run_simple(param.run_path, param.run_name)
if not simple_result:
    raise Exception("Simple failed")

Exception: Simple failed

In [ ]:
print(simple_result)